# LlamaIndex Bioconductor top 500 RAG model

This model is doing the 

In [7]:
import os, openai, requests, logging, sys, IProgress
import pandas as pd
from IPython.display import Markdown, display

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from llama_index.core import (
    SimpleDirectoryReader,
    StorageContext,
    VectorStoreIndex,
)

from llama_index.core.settings import Settings

from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

from llama_index.vector_stores.azureaisearch import (
    AzureAISearchVectorStore,
    IndexManagement,
    MetadataIndexFieldType,
)

from llama_index.core.query_engine import (
    CustomQueryEngine,
    RetrieverQueryEngine
)
from llama_index.core.retrievers import (
    BaseRetriever,
    VectorIndexRetriever
)

from llama_index.core.response_synthesizers import (
    BaseSynthesizer,
    ResponseMode
)

from llama_index.core import (
    PromptTemplate,
    VectorStoreIndex,
    get_response_synthesizer
)

from llama_index.core.postprocessor import SimilarityPostprocessor

In [8]:
aoai_api_key = os.getenv('AZURE_OPENAI_API_KEY')  # AZURE_OPENAI_API_KEY
aoai_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')  # AZURE_OPENAI_ENDPOINT
aoai_api_version = "2023-05-15"
aoai_embedding_model_name = "text-embedding-ada-002"
aoai_deployment_name="gpt-4-turbo"

llm = AzureOpenAI(
    model=aoai_deployment_name,
    deployment_name=aoai_deployment_name,
    api_key=aoai_api_key,
    azure_endpoint=aoai_endpoint,
    api_version=aoai_api_version,
)

# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model=aoai_embedding_model_name,
    deployment_name=aoai_embedding_model_name,
    api_key=aoai_api_key,
    azure_endpoint=aoai_endpoint,
    api_version=aoai_api_version,
)

In [9]:
search_service_api_key = os.getenv('OPENAI_SEARCH_API_KEY') # AZURE_AI_SEARCH_KEY
search_service_endpoint = os.getenv('AZURE_AI_SEARCH_ENDPOINT') # AZURE_AI_SEARCH_ENDPOINT
search_service_api_version = "2023-11-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-vector-top-500"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,   
    credential=credential,
)

# Use search client to demonstration using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [10]:
metadata_fields = {
    "author": "author",
    "theme": ("topic", MetadataIndexFieldType.STRING),
    "director": "director",
}

vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    filterable_metadata_field_keys=metadata_fields,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1536,
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
)

# Load the data - Bioc top 500 RAG dataset 

In [11]:
persist_index_dir = "/Users/niteshturaga/Documents/HMS/bioc-chat-bot-testing/llama_index/bioc_top_500"

top_500_dir = "/Users/niteshturaga/Documents/HMS/bioc-chat-bot-testing/data/bioc_3_18_top_500"

In [6]:
documents = SimpleDirectoryReader(top_500_dir).load_data()

Multiple definitions in dictionary at byte 0x1d8b for key /Group
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Multiple definitions in dictionary at byte 0x125d for key /Group


In [12]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [13]:
storage_context.persist(persist_dir=persist_index_dir)

Settings.llm = llm
Settings.embed_model = embed_model

index = VectorStoreIndex.from_documents(
    documents, 
    storage_context=storage_context,
    show_progress=True
)

Parsing nodes:   0%|          | 0/26094 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

...

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1520 [00:00<?, ?it/s]

In [14]:
index.storage_context.persist(persist_dir=persist_index_dir)

In [31]:
bioc_context_str = """\
Act as an expert in the R programming language and the Bioconductor suite \
of packages.  ​Your job is to advise users on the usage of the various \
Bioconductor packages considering the documents you have in the \
vector index store. To complete this task, you can use the data you have stored \
that contain the vignettes of the software packages in Bioconductor. \
​You may also answer some general R, general programming, or Biomedical \
information. If you do not know the answer ask the user to refer to \
https://bioconductor.org. Add a disclaimer at the end of each response  \
saying this response is AI generated, and should be independently verified \
by the user.
"""

qa_prompt_template = PromptTemplate(
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query.\n"
    "Query: {query_str}\n"
    "Answer: "
)

# configure retriever
retriever = VectorIndexRetriever(
    index=index, similarity_top_k=5
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="compact"
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)


class RAGStringQueryEngine(CustomQueryEngine):
    """RAG String Query Engine."""

    retriever: BaseRetriever
    response_synthesizer: BaseSynthesizer
    llm: AzureOpenAI
    qa_prompt: PromptTemplate

    def custom_query(self, query_str: str):
        nodes = self.retriever.retrieve(query_str)

        context_str = "\n\n".join([n.node.get_content() for n in nodes])
        response = self.llm.complete(
            qa_prompt_template.format(context_str=bioc_context_str, query_str=query_str)
        )
        return str(response)


In [32]:
query_engine = RAGStringQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    llm=llm,
    qa_prompt=qa_prompt_template,
)

In [33]:
def ask_rag(query):
    response = query_engine.query(query)
    display(Markdown(f"<b>{query}</b>"))
    display(Markdown(f"<b>{response}</b>"))
    display(Markdown(f"---------------------"))

In [34]:
df = pd.read_csv('bioc_qa_top10.csv')

In [35]:
ask_rag(df["Question"][1])

<b>I am working on RNA-Seq data. I'm using DESeq2 for my analysis. I have 20 samples from 3 batches. I am testing for 2 conditions, cond1 and cond2.dds <- DESeqDataSetFromMatrix(countData = countTable3, colData = coldata, design = ~cond1 * cond2). When i performed PCA, I could clearly see some batch effect. I read in the forum that adding batch to the design in DESeq removes the batch effect. But I am not sure if this is the right way to go about it because I can still see the same batch effect dds <-DESeqDataSetFromMatrix(countData = countTable3, colData = coldata, design = ~batch+cond1*cond2) I tried using Combat but I'm unable to use combat results in DESeq. It gives me the following error.Error in DESeqDataSet(se, design = design, ignoreRank) : some values in assay are negative . I am not sure if removeBatchEffects() function can be used with DESeq. Can someone please help me out here.</b>

<b>When dealing with batch effects in RNA-Seq data analysis using DESeq2, it is indeed common to include the batch variable in the design formula to account for the unwanted variation. Your approach to modify the design formula to include the batch effect is correct:

```r
dds <- DESeqDataSetFromMatrix(countData = countTable3, colData = coldata, design = ~batch + cond1 * cond2)
```

However, if you still observe batch effects after including the batch in the design, it might be due to the batch effect not being fully captured by the linear model or other complexities in the data. Here are a few steps you can take:

1. **Visualize Data**: Before and after including the batch in the design, use PCA or other clustering methods to visualize the data. This will help you assess the extent of the batch effect.

2. **sva Package**: Consider using the `sva` package to estimate surrogate variables for batch effects that might not be captured by the observed batch variable.

3. **Combat**: If you decide to use ComBat from the `sva` package, you should apply it to the raw count data before creating the `DESeqDataSet`. ComBat is designed to work on non-normalized, non-log-transformed counts. After correcting for batch effects, you can then proceed with DESeq2 analysis.

4. **Negative Values**: The error you're encountering with negative values suggests that the data might have been transformed or normalized in a way that's not compatible with DESeq2. DESeq2 expects raw counts as input, so ensure that the data you're using with ComBat has not been transformed or normalized.

5. **removeBatchEffect**: The `limma` package's `removeBatchEffect` function is typically used for visualization purposes and not for adjusting the data before differential expression analysis. It's more appropriate to include batch in the design formula as you have done.

6. **Consult Vignettes**: Check the DESeq2 vignette and other Bioconductor resources for advice on dealing with batch effects. The vignettes contain valuable examples and best practices.

Here's an example of how you might use ComBat before DESeq2:

```r
library(sva)
library(DESeq2)

# Assuming countTable3 is a matrix of raw counts and coldata contains the batch information
modCombat <- model.matrix(~batch + cond1 * cond2, coldata)
countTable3_corrected <- ComBat(dat=countTable3, batch=coldata$batch, mod=modCombat)

# Now create the DESeqDataSet with the batch-corrected data
dds <- DESeqDataSetFromMatrix(countData = countTable3_corrected, colData = coldata, design = ~cond1 * cond2)
```

Remember to consult the documentation and vignettes for each package for detailed instructions and examples. If you continue to encounter issues, the Bioconductor support forum is a valuable resource where you can ask specific questions about your analysis.

This response is AI-generated and should be independently verified by the user. For more detailed guidance, please refer to the official documentation and resources at https://bioconductor.org.</b>

---------------------

## Iterate over all the questions

In [23]:
## Loop through the questions and ask the RAG model for answers, the questions are in the column "Question", write the response in a new column "Response-RAG"
for index, row in df.iterrows():
    question = row['Question']
    response = ask_rag(question)


<b>I am a bit confused about the concepts of the 3 things: FDR, FDR adjusted p-value and q-value, which I initially thought I was clear about. Are FDR adjusted p-value the same as q-value? (my understanding is that FDR adjusted p-value = original p-value * number of genes/rank of the gene, is that right?) When people say xxx genes are differentially expressed with an FDR cutoff of 0.05, does that mean xxx genes have an FDR adjusted p-value smaller than 0.05?</b>

<b>The concepts of FDR, FDR adjusted p-values, and q-values are related but have distinct meanings in the context of multiple hypothesis testing, which is common in bioinformatics and genomics research.

1. **FDR (False Discovery Rate)**: This is the expected proportion of false discoveries (incorrectly rejected null hypotheses) among all discoveries (rejected null hypotheses). For example, if you perform 1000 hypothesis tests and you expect 5% of the significant results to be false discoveries, you would be working with an FDR of 0.05.

2. **FDR adjusted p-value**: This is not a standard term, but it seems to refer to p-values that have been adjusted for multiple comparisons to control the FDR. These adjusted p-values are often called q-values, but the term "FDR adjusted p-value" might be used informally by some researchers.

3. **q-value**: This is a specific type of adjusted p-value that provides a measure of the minimum FDR at which a particular hypothesis test would be considered significant. The q-value is a more direct measure of the FDR and is often preferred in genomics studies where many tests are conducted simultaneously.

Your understanding of the FDR adjusted p-value as the original p-value multiplied by the number of genes divided by the rank of the gene is a simplification of one method of adjustment, the Benjamini-Hochberg procedure. This procedure controls the FDR under certain conditions. However, the actual calculation of q-values or FDR adjusted p-values can be more complex and depends on the specific statistical method used.

When people say "xxx genes are differentially expressed with an FDR cutoff of 0.05," they typically mean that xxx genes have q-values (or FDR adjusted p-values) that are smaller than 0.05. This implies that, on average, no more than 5% of the genes declared differentially expressed are expected to be false discoveries.

In Bioconductor, packages such as `limma` or `DESeq2` can be used to perform differential expression analysis and calculate q-values for the purpose of controlling the FDR. You can refer to the vignettes of these packages for detailed explanations and examples of how to interpret the results of such analyses.

Please note that this response is AI-generated and should be independently verified by the user. For more detailed information, you can refer to the Bioconductor website at https://bioconductor.org.</b>

---------------------

<b>I am working on RNA-Seq data. I'm using DESeq2 for my analysis. I have 20 samples from 3 batches. I am testing for 2 conditions, cond1 and cond2.dds <- DESeqDataSetFromMatrix(countData = countTable3, colData = coldata, design = ~cond1 * cond2). When i performed PCA, I could clearly see some batch effect. I read in the forum that adding batch to the design in DESeq removes the batch effect. But I am not sure if this is the right way to go about it because I can still see the same batch effect dds <-DESeqDataSetFromMatrix(countData = countTable3, colData = coldata, design = ~batch+cond1*cond2) I tried using Combat but I'm unable to use combat results in DESeq. It gives me the following error.Error in DESeqDataSet(se, design = design, ignoreRank) : some values in assay are negative . I am not sure if removeBatchEffects() function can be used with DESeq. Can someone please help me out here.</b>

<b>When dealing with batch effects in RNA-Seq data analysis using DESeq2, it's crucial to correctly incorporate the batch variable into your analysis to minimize its impact on the results. Your approach to include the batch effect in the design formula is generally correct. The design formula `~batch + cond1*cond2` tells DESeq2 to account for batch effects alongside the interaction between your two conditions. However, seeing the same batch effect in PCA after this adjustment suggests that the batch effect might not be fully accounted for or that there are other confounding factors.

Here are a few steps and considerations to help you address this issue:

1. **Ensure Correct Coding of Batch Variable**: Make sure that the `batch` variable in your `colData` is correctly coded. It should accurately reflect the batch each sample belongs to.

2. **Visualization**: After adjusting for batch effects in your DESeq2 analysis, it's a good practice to visualize the data again using PCA or other dimensionality reduction techniques to assess if the batch effect persists. Sometimes, visual inspection might suggest a batch effect even when its impact on differential expression results is minimized.

3. **Using `removeBatchEffects` from limma**: While `removeBatchEffects` is not part of the DESeq2 workflow, it's commonly used for visualization purposes rather than adjusting the data before differential expression analysis. You can use it to adjust your PCA plots or other visualizations post-analysis to assess the batch effect visually. However, this does not change the underlying count data used in DESeq2.

4. **ComBat**: If you've attempted to use ComBat (from the sva package) to correct for batch effects before running DESeq2 and encountered negative values, it's likely because ComBat is adjusting the data in a way that's incompatible with DESeq2's expectations. DESeq2 expects non-negative integer counts, but ComBat might output corrected values that are not integers or are negative. It's generally recommended to incorporate batch effects directly in the DESeq2 design formula rather than pre-adjusting counts with ComBat.

5. **Alternative Strategies**: If batch effects are still a concern, consider additional strategies such as:
   - Re-evaluating the experimental design to ensure that batches are not confounded with your conditions of interest.
   - Exploring other normalization or batch correction methods that are compatible with count data and DESeq2's requirements.
   - Consulting with a statistician or bioinformatician with experience in batch effect correction for complex experimental designs.

Remember, handling batch effects is a critical step in ensuring the validity of your RNA-Seq analysis results. The approach should be tailored to your specific dataset and experimental design.

This response is AI-generated and should be independently verified by the user. For further guidance, refer to the DESeq2 documentation and the Bioconductor project website at https://bioconductor.org.</b>

...

<b>It sounds like you're encountering a common issue in bioinformatics where not all Ensembl IDs can be directly mapped to gene symbols using a single database or tool, such as AnnotationDbi. This discrepancy can arise due to several reasons, including differences in database versions, updates in gene annotations, or the presence of novel or less-characterized genes that might not be fully annotated in all databases.

Here are a few steps and alternatives you can consider to improve your gene symbol conversion rate:

1. **Ensure Database Compatibility**: Make sure that the version of the AnnotationDbi package and its underlying databases (like org.Hs.eg.db for human) are compatible with the version of the Ensembl IDs you are using. Mismatches in versions can lead to missing annotations.

2. **Use the latest version of Bioconductor and AnnotationDbi**: Updating your packages can sometimes resolve these issues as databases are continuously updated.

3. **biomaRt Alternative**: Since you've had success with biomaRt, consider using it within your R workflow for the conversion process. biomaRt directly queries the Ensembl database and might provide more up-to-date or comprehensive mappings. Here's a simple example of how you might use biomaRt for this purpose:

```R
library(biomaRt)
ensembl <- useMart("ensembl", dataset = "hsapiens_gene_ensembl")
genes <- c("ENSG00000139618", "ENSG00000155657") # Example Ensembl IDs
genes_mapped <- getBM(attributes = c('ensembl_gene_id', 'hgnc_symbol'), 
                      filters = 'ensembl_gene_id', 
                      values = genes, 
                      mart = ensembl)
```

4. **Check for Alternative Gene Symbols**: Sometimes, an Ensembl ID might be mapped to an alternative gene symbol that you didn't expect. Ensure that you're checking all possible mappings.

5. **Manual Verification for Critical Genes**: For significantly differentially expressed genes that are of particular interest, manual verification using the Ensembl website or literature search might be necessary.

6. **Community and Support**: If you continue to face issues, consider reaching out to the Bioconductor support site or forums. Sharing your specific code and examples can help the community provide more targeted advice.

Remember, gene annotation is a dynamic field, and discrepancies between databases are not uncommon. It's always a good practice to document the versions of databases and tools used in your analysis for reproducibility and to facilitate troubleshooting.

This response is AI-generated and should be independently verified by the user.</b>

---------------------

<b>How do I merge a list of GRanges? What I want is a GRanges object containing the union of all the ranges and the metadata in the list. I could loop over the list, taking the union of each pair of GRanges, but the union loses the metadata. Is there a way to merge all my GRanges into one giant GRanges, with metadata? What I ultimately want is a data.frame of counts so that I can input it into limma.</b>

<b>To merge a list of `GRanges` objects into a single `GRanges` object while preserving metadata, you can use the `reduce` function from the GenomicRanges package, part of the Bioconductor suite. However, `reduce` alone will not preserve metadata. To address this, you can first concatenate all your `GRanges` objects into one large `GRanges` object using the `c` function, which will preserve the metadata. Then, you can use `reduce` on this combined object to merge overlapping or adjacent ranges. Here's a step-by-step guide:

1. **Concatenate GRanges**: Use the `c()` function to concatenate all your `GRanges` objects into one. This step ensures that all metadata is kept in the combined object.

```R
library(GenomicRanges)

# Assuming your list of GRanges objects is named gr_list
combinedGR <- do.call(c, gr_list)
```

2. **Reduce Combined GRanges**: Now, use the `reduce` function on the combined `GRanges` object. This will merge overlapping and adjacent ranges. Note that `reduce` will not keep the metadata for the merged ranges.

```R
reducedGR <- reduce(combinedGR)
```

3. **Handling Metadata**: If you need to preserve or handle metadata in a specific way (e.g., summarizing, taking the union, etc.), you should do this before or after the `reduce` step, depending on what makes sense for your specific use case. For example, you might aggregate metadata before reducing if you want to summarize it across merged ranges.

4. **Converting to Data.frame for limma**: Once you have your final `GRanges` object, you can convert it to a `data.frame` using the `as.data.frame()` function. This `data.frame` can then be used as input for differential expression analysis with limma.

```R
finalDF <- as.data.frame(reducedGR)
```

Now, you have a `data.frame` that you can use with limma for your analysis.

Remember, the specific steps, especially regarding metadata handling, might need to be adjusted based on the exact nature of your metadata and what you need to retain for your analysis with limma.

This response is AI-generated and should be independently verified by the user.</b>

---------------------

# Try to get the index from the storage context 

In [24]:
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.storage.index_store import SimpleIndexStore
from llama_index.core.vector_stores import SimpleVectorStore

storage_context_from_store = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir=persist_index_dir)
)


In [27]:
from llama_index.core import (
    load_index_from_storage,
    load_indices_from_storage,
    load_graph_from_storage,
)

index = load_index_from_storage(storage_context)


### We can now redo the same analysis again using the RAG model with the stored indices.